In [1]:

# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:
# a_pattern = 'Si/SI2018/SI11318.Chn'
# 2022-nov-16: outros espectros:
# a_pattern = "Filtros/2022/Cci/CCI1603-I.Chn"
# a_pattern = "Filtros/2022/Cci/CCI2302-I.Chn"

# a_pattern = "Eso_non_existe.Chn"

# 2022-Dez-21 Buscando outro espectro
# a_pattern = 'CCI1603-I'
# 2022-Dez-28 Este é um espectro mais recente:
# a_pattern = 'SI09722.Chn'
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
# a_pattern = 'SI06322.Chn'
# a_pattern = 'SI06122.Chn'

In [2]:
import numpy as np
from numpy.polynomial import Polynomial as P  # 2020-09-06 Esta é a nova classe recomendada
from sklearn import linear_model

import pandas as pd
from random import randrange
# import plotly.graph_objects as go

In [3]:
from ograyspy_class import Ograyspy, select_spectrum_from_folder_list
from spec_class import Spec

In [4]:
ogra = Ograyspy()
print(ogra.info_node)
# to_be_found = 'Genie_Transfer'
to_be_found = 'some_spectra' # NAO ESTAH ACHANDO localmente no note!

Found folder name:  /home/jovyan/.bashrc
Parts:  ('/', 'home', 'jovyan', '.bashrc')
No. spec files:  0
jupyter-terrabela-2dograyspy-2dcn6uf97b


In [5]:
print('\nExec ogra.define_files_folder(to_be_found):')
ogra.define_files_folder(to_be_found)
print(ogra.pkl_folder_files)
spectra_list_df = pd.read_pickle(ogra.pkl_folder_files)
del ogra


Exec ogra.define_files_folder(to_be_found):
Found folder name:  /home/jovyan/data/some_spectra
Parts:  ('/', 'home', 'jovyan', 'data', 'some_spectra')
No. spec files:  12
Linuxjupyter-terrabela-2dograyspy-2dcn6uf97b.pkl


In [6]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [7]:
reduc_nms

['CCI0902-I.Chn',
 'CCI0202-I.Chn',
 'CCI0903-I.Chn',
 'SI06122.Chn',
 'CCI0501-I.Chn',
 'CCI2302-I.Chn',
 'CCI1201-I.Chn',
 'CCI1503-F.Chn',
 'CCI1901-I.Chn',
 'CCI1602-I.Chn',
 'CCI2601-I.Chn',
 'CCI1603-I.Chn']

In [8]:
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
a_pattern = 'SI06122.Chn'

In [9]:
f_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)
f_name, reduced_f_name

Existing:
Achou! indice=3, nomearq = SI06122.Chn
name:  /home/jovyan/data/some_spectra/SI06122.Chn
Final choices:
spectra_path: /home/jovyan/data/some_spectra
a_spec_name: /home/jovyan/data/some_spectra/SI06122.Chn
reduced_f_name: SI06122.Chn


(PosixPath('/home/jovyan/data/some_spectra/SI06122.Chn'), 'SI06122.Chn')

In [10]:
nucl_iear1_df = pd.read_pickle('nucl_iear1_list.pkl')
nucl_iear1_df

,nuclide_name,energy,unc_en,intensity,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma
1,24na,1368.6250,0.005,99.994,0.002,14.956,B-,100.0,1368.6,True
2,24na,2754.0080,0.011,99.867,0.01,14.956,B-,100.0,1368.6,False
9,51cr,320.0824,0.0004,9.910,0.01,27.704,EC,100.0,320.1,True
15,54mn,834.8480,0.003,99.976,0.001,312.200,EC+B+,100.0,834.8,True
22,58co,810.7593,0.002,99.450,0.01,70.860,EC+B+,100.0,810.8,True
...,...,...,...,...,...,...,...,...,...,...
1066,228Ac,89.9540,,2.010,0.19,6.150,B-,100.0,911.6,False
1067,228Ac,93.3470,,3.200,0.3,6.150,B-,100.0,911.6,False
1068,228Ac,105.5660,,1.160,0.11,6.150,B-,100.0,911.6,False
1069,228Ac,106.8940,,1.540,0.15,6.150,B-,100.0,911.6,False


In [11]:
# 2023-Jun-15: Setting gamma lines/ranges to dismiss in the analysis
df1 = nucl_iear1_df
df1["is_to_consider"] = True
df1.loc[((df1.energy > 509) & (df1.energy < 513)) | (df1.energy < 100), "is_to_consider"] = False
del df1
nucl_iear1_df

,nuclide_name,energy,unc_en,intensity,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma,is_to_consider
1,24na,1368.6250,0.005,99.994,0.002,14.956,B-,100.0,1368.6,True,True
2,24na,2754.0080,0.011,99.867,0.01,14.956,B-,100.0,1368.6,False,True
9,51cr,320.0824,0.0004,9.910,0.01,27.704,EC,100.0,320.1,True,True
15,54mn,834.8480,0.003,99.976,0.001,312.200,EC+B+,100.0,834.8,True,True
22,58co,810.7593,0.002,99.450,0.01,70.860,EC+B+,100.0,810.8,True,True
...,...,...,...,...,...,...,...,...,...,...,...
1066,228Ac,89.9540,,2.010,0.19,6.150,B-,100.0,911.6,False,False
1067,228Ac,93.3470,,3.200,0.3,6.150,B-,100.0,911.6,False,False
1068,228Ac,105.5660,,1.160,0.11,6.150,B-,100.0,911.6,False,True
1069,228Ac,106.8940,,1.540,0.15,6.150,B-,100.0,911.6,False,True


In [12]:
a_spec = Spec(f_name, reduced_f_name)

In [13]:
dir(a_spec)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'channel_energy_calib',
 'curr_h_win',
 'det_descr',
 'f_name',
 'final_composed_baseline',
 'generate_pandas_dataframe',
 'identify_nuclides',
 'lv_time',
 'net_spec_ser_an',
 'nucl_an',
 'origin_spec_ser_an',
 'pkl_file',
 'reduced_f_name',
 'rl_time',
 'sam_descr',
 'source_datetime',
 'spec_io',
 'start_datetime',
 'sufx',
 'total_analysis']

In [14]:
orig_ser = a_spec.origin_spec_ser_an
vars(orig_ser)

{'mix_regions': None,
 'spl_baseline': None,
 'eval_baseline': None,
 'y_s': array([0, 0, 0, ..., 3, 4, 0]),
 'nzero': array([False, False, False, ...,  True,  True, False]),
 'n_ch': 4096,
 'final_baseline': None,
 'x_s': array([0.000e+00, 1.000e+00, 2.000e+00, ..., 4.093e+03, 4.094e+03,
        4.095e+03]),
 'chans_nzero': array([  26.,   27.,   28., ..., 4092., 4093., 4094.]),
 'counts_nzero': array([  1,  81, 797, ...,   4,   3,   4]),
 'unc_y': array([ 1.        ,  9.        , 28.23118843, ...,  2.        ,
         1.73205081,  2.        ]),
 'given_variance': array([0, 0, 0, ..., 3, 4, 0]),
 'y_smoothed': array([0, 0, 0, ..., 3, 4, 0]),
 'fft_s': None,
 'is_reg': array([False, False, False, ..., False, False, False]),
 'xs_all_mplets': [],
 'ys_all_mplets': [],
 'ys_all_steps': [],
 'chans_in_multiplets_list': [],
 'calculated_step_counts': [],
 'xs_bl_in_reg': array([], dtype=float64),
 'ys_bl_in_reg': array([], dtype=float64),
 'xs_bl_out_reg': array([], dtype=float64),
 'ys_b

In [15]:
a_spec.total_analysis(gener_dataframe=True)
# A opção acima, gener_dataframe=True,  grava o pkl_file do espectro
vars(a_spec)

Starting Spec.total_analysis...
resolve_peaks_and_regions:
define_multiplets_regions completado. Define: self.mix_regions.
resolve_peaks_and_regions:
define_multiplets_regions completado. Define: self.mix_regions.
Finish Spec.total_analysis!


{'net_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x7fc88de292d0>,
 'final_composed_baseline': <generic_series_analysis_class.GenericSeriesAnalysis at 0x7fc88de29780>,
 'f_name': PosixPath('/home/jovyan/data/some_spectra/SI06122.Chn'),
 'reduced_f_name': 'SI06122.Chn',
 'sufx': '.chn',
 'spec_io': None,
 'lv_time': 50000.0,
 'rl_time': 50028.96,
 'source_datetime': None,
 'pkl_file': PosixPath('/home/jovyan/data/some_spectra/SI06122.pkl'),
 'origin_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x7fc88de296f0>,
 'start_datetime': datetime.datetime(2022, 7, 27, 16, 56, 29),
 'det_descr': 'MAQUINA_01 MCB 1 Input 1',
 'sam_descr': 'SI-061/22 Agua da Piscina Reator IEA-R1 (GMX)',
 'channel_energy_calib': <genericcalib_class.ChannelEnergyCalib at 0x7fc88de29b10>,
 'nucl_an': <nuclide_analysis_class.NuclideAnalysis at 0x7fc88de2b130>}

In [16]:
# vars(orig_ser)

In [17]:
net_ser = a_spec.net_spec_ser_an
vars(net_ser)

{'mix_regions': array([[  26,   35],
        [ 122,  135],
        [ 140,  151],
        [ 230,  243],
        [ 389,  404],
        [ 486,  493],
        [ 576,  587],
        [ 711,  722],
        [ 832,  851],
        [ 954,  967],
        [ 997, 1018],
        [1076, 1094],
        [1110, 1121],
        [1157, 1170],
        [1184, 1195],
        [1250, 1263],
        [1327, 1340],
        [1363, 1382],
        [1447, 1462],
        [1490, 1507],
        [1533, 1548],
        [1577, 1599],
        [1824, 1846],
        [1919, 1934],
        [2027, 2038],
        [2179, 2196],
        [2241, 2254],
        [2255, 2281],
        [2389, 2406],
        [2464, 2477],
        [2600, 2622],
        [2833, 2840],
        [2886, 2903],
        [3026, 3035],
        [3445, 3456],
        [3606, 3625]]),
 'spl_baseline': None,
 'eval_baseline': None,
 'y_s': array([0.        , 0.        , 0.        , ..., 0.        , 1.87084501,
        0.        ]),
 'nzero': array([False, False, False, ...,

In [18]:
print(a_spec.pkl_file)

/home/jovyan/data/some_spectra/SI06122.pkl


In [19]:
a_spec.pkl_file

PosixPath('/home/jovyan/data/some_spectra/SI06122.pkl')

In [20]:
# Só para verificar se o pkl_file pode ser recuperado:
read_analysis_spec = pd.read_pickle(a_spec.pkl_file)

In [21]:
read_orig = read_analysis_spec['origin_spec_ser_an'][0]
orig_ser = read_orig
# vars(orig_ser)

In [22]:
read_net = read_analysis_spec['net_spec_ser_an'][0]
net_ser = read_net
vars(net_ser)

{'mix_regions': array([[  26,   35],
        [ 122,  135],
        [ 140,  151],
        [ 230,  243],
        [ 389,  404],
        [ 486,  493],
        [ 576,  587],
        [ 711,  722],
        [ 832,  851],
        [ 954,  967],
        [ 997, 1018],
        [1076, 1094],
        [1110, 1121],
        [1157, 1170],
        [1184, 1195],
        [1250, 1263],
        [1327, 1340],
        [1363, 1382],
        [1447, 1462],
        [1490, 1507],
        [1533, 1548],
        [1577, 1599],
        [1824, 1846],
        [1919, 1934],
        [2027, 2038],
        [2179, 2196],
        [2241, 2254],
        [2255, 2281],
        [2389, 2406],
        [2464, 2477],
        [2600, 2622],
        [2833, 2840],
        [2886, 2903],
        [3026, 3035],
        [3445, 3456],
        [3606, 3625]]),
 'spl_baseline': None,
 'eval_baseline': None,
 'y_s': array([0.        , 0.        , 0.        , ..., 0.        , 1.87084501,
        0.        ]),
 'nzero': array([False, False, False, ...,

In [23]:
# 2023-May-26
# Ok! Espectro gravado, depois lido, então vamos prosseguir
# com a identificação dos nuclídeos.
# Agora, posso ler e analisar um espectro, gravá-lo e depois, em outro momento,
# ler o pkl com a análise.

In [24]:
x_nz = orig_ser.chans_nzero
y_nz = orig_ser.counts_nzero

In [25]:
chans = net_ser.x_s
ys_net_counts = net_ser.y_s
peaks_net = net_ser.pk_parms.peaks
peaks_orig = orig_ser.pk_parms.peaks
counts = orig_ser.y_s
chans_nzero = orig_ser.chans_nzero
counts_nzero = orig_ser.counts_nzero
unc_y = orig_ser.unc_y
eval_y = orig_ser.y_smoothed
eval_bl = orig_ser.eval_baseline
fin_bl = orig_ser.final_baseline
inis = orig_ser.pk_parms.propts['left_bases']
fins = orig_ser.pk_parms.propts['right_bases']

In [26]:
# 2023-Jun-15
# CRUCIAL step: take the dict net_ser.pk_parms' keys/values and organize them as a pd.Dataframe
vars_pkprms = vars(net_ser.pk_parms)
keys_to_get = ['peaks', 'fwhm_centr', 'rough_sums', 'centroids', 'variances']
prep_for_dict = [(key, vars_pkprms[key]) for key in keys_to_get]
pks_dict = dict(prep_for_dict)
peaks_df = pd.DataFrame.from_dict(pks_dict)
peaks_df

,peaks,fwhm_centr,rough_sums,centroids,variances
0,30,29.785567,1893.942368,29.963695,14133.214332
1,128,128.364887,2876.788495,127.767831,11283.211505
2,145,144.459911,1278.607977,144.896569,10161.329960
3,236,235.273124,462.759320,235.202199,12019.150830
4,396,396.401754,630.310441,396.426778,10404.641509
5,489,488.997063,249.976688,488.967375,4072.023312
6,581,581.463638,430.396173,581.647407,5405.821287
7,716,715.508125,222.738094,715.666805,4033.829966
8,841,841.967813,2900.206925,841.976062,7873.573309
9,960,960.088168,483.773366,960.065118,3335.841582


In [27]:
pr_pk = dict([('pk_hei',net_ser.pk_parms.propts['peak_heights']),
              ('lf_thr',net_ser.pk_parms.propts['left_thresholds']),
              ('rg_thr',net_ser.pk_parms.propts['right_thresholds']),
              ('promns',net_ser.pk_parms.propts['prominences']),
              ('lf_bas',net_ser.pk_parms.propts['left_bases']),
              ('rg_bas',net_ser.pk_parms.propts['right_bases']),
              ('widths',net_ser.pk_parms.propts['widths']),
              ('wi_hei',net_ser.pk_parms.propts['width_heights']),
              ('lf_ips',net_ser.pk_parms.propts['left_ips']),
              ('rg_ips',net_ser.pk_parms.propts['right_ips'])])

In [28]:
pks_properties_df = pd.DataFrame(pr_pk)
pks_properties_df

,pk_hei,lf_thr,rg_thr,promns,lf_bas,rg_bas,widths,wi_hei,lf_ips,rg_ips
0,818.646195,201.465549,450.836680,818.646195,28,33,2.244704,409.323097,28.663214,30.907919
1,720.585627,359.256221,71.539300,720.585627,100,137,2.761022,360.292813,126.984376,129.745398
2,389.571554,4.276963,247.018556,389.571554,142,151,2.657272,194.785777,143.131275,145.788547
3,123.425784,40.769738,84.205044,123.425784,233,240,2.919528,61.712892,233.813360,236.732888
4,162.845302,70.909899,49.239266,162.845302,392,404,3.333009,81.422651,394.735249,398.068258
5,125.089174,88.241458,88.978919,125.089174,485,493,1.411704,62.544587,488.291211,489.702915
6,147.276132,89.934593,1.069452,147.276132,579,585,2.564867,73.638066,580.181204,582.746071
7,79.006167,15.639623,46.454617,79.006167,713,719,2.684467,39.503084,714.165892,716.850359
8,559.895371,213.926360,11.088422,559.895371,836,850,4.639239,279.947685,839.648194,844.287433
9,148.624963,51.372759,40.677023,148.624963,955,963,2.878815,74.312482,958.648761,961.527576


In [29]:
pks_comprehensive_df = pd.concat([peaks_df, pks_properties_df],axis=1)
pks_comprehensive_df

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,rg_bas,widths,wi_hei,lf_ips,rg_ips
0,30,29.785567,1893.942368,29.963695,14133.214332,818.646195,201.465549,450.836680,818.646195,28,33,2.244704,409.323097,28.663214,30.907919
1,128,128.364887,2876.788495,127.767831,11283.211505,720.585627,359.256221,71.539300,720.585627,100,137,2.761022,360.292813,126.984376,129.745398
2,145,144.459911,1278.607977,144.896569,10161.329960,389.571554,4.276963,247.018556,389.571554,142,151,2.657272,194.785777,143.131275,145.788547
3,236,235.273124,462.759320,235.202199,12019.150830,123.425784,40.769738,84.205044,123.425784,233,240,2.919528,61.712892,233.813360,236.732888
4,396,396.401754,630.310441,396.426778,10404.641509,162.845302,70.909899,49.239266,162.845302,392,404,3.333009,81.422651,394.735249,398.068258
5,489,488.997063,249.976688,488.967375,4072.023312,125.089174,88.241458,88.978919,125.089174,485,493,1.411704,62.544587,488.291211,489.702915
6,581,581.463638,430.396173,581.647407,5405.821287,147.276132,89.934593,1.069452,147.276132,579,585,2.564867,73.638066,580.181204,582.746071
7,716,715.508125,222.738094,715.666805,4033.829966,79.006167,15.639623,46.454617,79.006167,713,719,2.684467,39.503084,714.165892,716.850359
8,841,841.967813,2900.206925,841.976062,7873.573309,559.895371,213.926360,11.088422,559.895371,836,850,4.639239,279.947685,839.648194,844.287433
9,960,960.088168,483.773366,960.065118,3335.841582,148.624963,51.372759,40.677023,148.624963,955,963,2.878815,74.312482,958.648761,961.527576


In [30]:
def add_engy_to_pks_df(peaks_net_kev_df, func_en):
    peaks_net_kev_df['engy_pk_det'] = func_en(peaks_net_kev_df.centroids)

In [31]:
add_engy_to_pks_df(pks_comprehensive_df, a_spec.channel_energy_calib.get_energy)

In [32]:
pks_comprehensive_df

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,rg_bas,widths,wi_hei,lf_ips,rg_ips,engy_pk_det
0,30,29.785567,1893.942368,29.963695,14133.214332,818.646195,201.465549,450.836680,818.646195,28,33,2.244704,409.323097,28.663214,30.907919,16.318529
1,128,128.364887,2876.788495,127.767831,11283.211505,720.585627,359.256221,71.539300,720.585627,100,137,2.761022,360.292813,126.984376,129.745398,76.080802
2,145,144.459911,1278.607977,144.896569,10161.329960,389.571554,4.276963,247.018556,389.571554,142,151,2.657272,194.785777,143.131275,145.788547,86.547152
3,236,235.273124,462.759320,235.202199,12019.150830,123.425784,40.769738,84.205044,123.425784,233,240,2.919528,61.712892,233.813360,236.732888,141.727537
4,396,396.401754,630.310441,396.426778,10404.641509,162.845302,70.909899,49.239266,162.845302,392,404,3.333009,81.422651,394.735249,398.068258,240.242260
5,489,488.997063,249.976688,488.967375,4072.023312,125.089174,88.241458,88.978919,125.089174,485,493,1.411704,62.544587,488.291211,489.702915,296.788299
6,581,581.463638,430.396173,581.647407,5405.821287,147.276132,89.934593,1.069452,147.276132,579,585,2.564867,73.638066,580.181204,582.746071,353.419539
7,716,715.508125,222.738094,715.666805,4033.829966,79.006167,15.639623,46.454617,79.006167,713,719,2.684467,39.503084,714.165892,716.850359,435.310799
8,841,841.967813,2900.206925,841.976062,7873.573309,559.895371,213.926360,11.088422,559.895371,836,850,4.639239,279.947685,839.648194,844.287433,512.490852
9,960,960.088168,483.773366,960.065118,3335.841582,148.624963,51.372759,40.677023,148.624963,955,963,2.878815,74.312482,958.648761,961.527576,584.648031


In [33]:
nucl_iear1_df

,nuclide_name,energy,unc_en,intensity,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma,is_to_consider
1,24na,1368.6250,0.005,99.994,0.002,14.956,B-,100.0,1368.6,True,True
2,24na,2754.0080,0.011,99.867,0.01,14.956,B-,100.0,1368.6,False,True
9,51cr,320.0824,0.0004,9.910,0.01,27.704,EC,100.0,320.1,True,True
15,54mn,834.8480,0.003,99.976,0.001,312.200,EC+B+,100.0,834.8,True,True
22,58co,810.7593,0.002,99.450,0.01,70.860,EC+B+,100.0,810.8,True,True
...,...,...,...,...,...,...,...,...,...,...,...
1066,228Ac,89.9540,,2.010,0.19,6.150,B-,100.0,911.6,False,False
1067,228Ac,93.3470,,3.200,0.3,6.150,B-,100.0,911.6,False,False
1068,228Ac,105.5660,,1.160,0.11,6.150,B-,100.0,911.6,False,True
1069,228Ac,106.8940,,1.540,0.15,6.150,B-,100.0,911.6,False,True


In [34]:
nucl_iear1_selctd_gamms_df = nucl_iear1_df.loc [
    (nucl_iear1_df.intensity > 1.0) & nucl_iear1_df.is_to_consider
]
nucl_iear1_selctd_gamms_df

,nuclide_name,energy,unc_en,intensity,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma,is_to_consider
1,24na,1368.6250,0.005,99.994,0.002,14.956,B-,100.0,1368.6,True,True
2,24na,2754.0080,0.011,99.867,0.01,14.956,B-,100.0,1368.6,False,True
9,51cr,320.0824,0.0004,9.910,0.01,27.704,EC,100.0,320.1,True,True
15,54mn,834.8480,0.003,99.976,0.001,312.200,EC+B+,100.0,834.8,True,True
22,58co,810.7593,0.002,99.450,0.01,70.860,EC+B+,100.0,810.8,True,True
...,...,...,...,...,...,...,...,...,...,...,...
944,228Ac,968.9710,0.017,15.800,0.3,6.150,B-,100.0,911.6,False,True
1014,228Ac,1588.2000,0.03,3.220,0.08,6.150,B-,100.0,911.6,False,True
1017,228Ac,1630.6270,0.01,1.510,0.04,6.150,B-,100.0,911.6,False,True
1068,228Ac,105.5660,,1.160,0.11,6.150,B-,100.0,911.6,False,True


In [35]:
118*42

4956

In [36]:
# Primeiro fiz assim:
# cross_df = pd.merge(pks_comprehensive_df, nucl_iear1_selctd_gamms_df, how='cross')
# cross_df["delta_en"] = cross_df.engy_pk_det - cross_df.energy
# cross_df

# Agora fazer de novo:
# cross_df_2 = pd.merge(matching_peaks_df, nucl_iear1_selctd_gamms_df, how='cross', suffixes=('_x', None))
# cross_df_2.drop(cross_df_2.filter(regex='_x$').columns, axis=1, inplace=True)
# cross_df_2["delta_en"] = cross_df_2.recalib_engy_ransac - cross_df_2.energy
# cross_df_2

In [37]:
def create_matching_peaks_df(pks_df, nucl_df, en_toler, must_be_key_gamma=False):
    if must_be_key_gamma:
        nucl_aux_df = pd.DataFrame(nucl_df.loc[nucl_df.is_key_gamma])
    else:
        nucl_aux_df = pd.DataFrame(nucl_df)
    cross_df = pd.merge(pks_df, nucl_aux_df, how='cross')
    cross_df["delta_en"] = cross_df.engy_pk_det - cross_df.energy
    aux_2 = pd.DataFrame(cross_df.loc[np.abs(cross_df.delta_en) < en_toler])
    X_energy = np.array(aux_2.energy).reshape(-1,1)
    y_delta_en = np.array(aux_2.delta_en)
    # Robustly fit linear model with RANSAC algorithm
    ransac = linear_model.RANSACRegressor()
    ransac.fit(X_energy, y_delta_en)
    inlier_mask = ransac.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)
    aux_2["recalib_engy_ransac"] = aux_2.engy_pk_det - ransac.predict(X_energy)
    aux_2["new_delta_en"] = aux_2.recalib_engy_ransac - aux_2.energy
    aux_3 = aux_2.loc[abs(aux_2.new_delta_en) < 0.4]
    new_ch_en_calib = P.fit(aux_3.centroids, aux_3.energy, deg=2)
    return aux_3, new_ch_en_calib

In [38]:
# cross_df = pd.merge(pks_comprehensive_df, nucl_iear1_selctd_gamms_df, how='cross')
 #    cross_df["delta_en"] = cross_df.engy_pk_det - cross_df.energy
  #  cross_df 

In [39]:
en_toler_calib = 3.0
en_toler_ident = 0.4

In [40]:
aux_3, new_ch_en_calib = create_matching_peaks_df(pks_comprehensive_df, nucl_iear1_selctd_gamms_df, en_toler_calib, must_be_key_gamma=False)
aux_3

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,...,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma,is_to_consider,delta_en,recalib_engy_ransac,new_delta_en
543,396,396.401754,630.310441,396.426778,10404.641509,162.845302,70.909899,49.239266,162.845302,392,...,0.5307178534777213,1.062200e+01,B-,100.000,238.6,True,True,1.610260,238.910731,0.278731
667,489,488.997063,249.976688,488.967375,4072.023312,125.089174,88.241458,88.978919,125.089174,485,...,0.11249527437185973,2.706000e+01,B-,100.000,351.9,False,True,1.564299,295.429188,0.205188
786,581,581.463638,430.396173,581.647407,5405.821287,147.276132,89.934593,1.069452,147.276132,579,...,0.2352736185805795,2.706000e+01,B-,100.000,351.9,True,True,1.487539,352.032790,0.100790
1129,960,960.088168,483.773366,960.065118,3335.841582,148.624963,51.372759,40.677023,148.624963,955,...,0.3,3.053000e+00,B-,100.000,583.2,True,True,1.461031,583.148572,-0.038428
1260,1003,1002.888165,769.886444,1002.931019,3458.113556,220.739274,40.375893,77.703349,220.739274,990,...,0.19,1.971000e+01,B-,99.979,609.3,True,True,1.519826,609.328630,0.007630
1428,1082,1082.208995,2125.483990,1082.200436,4570.516010,657.235129,251.432911,123.634783,657.235129,1073,...,0.09,2.498300e+02,B-,98.670,657.8,True,True,1.517639,657.741834,-0.018166
1566,1088,1088.722180,401.081225,1088.105023,2556.918775,97.223236,59.126652,28.889179,69.221096,1086,...,0.2,3.008000e+01,B-,100.000,661.7,True,True,1.228580,661.347876,-0.309124
1665,1115,1114.774649,176.695976,1115.022855,1999.057501,67.734059,24.189260,36.845929,67.734059,1112,...,0.05,2.498300e+02,B-,98.670,657.8,False,True,1.711762,677.787977,0.166277
1785,1163,1162.050038,336.295814,1161.943753,2591.560129,85.795902,0.176165,73.885618,85.795902,1152,...,0.07,2.498300e+02,B-,98.670,657.8,False,True,1.328023,706.444378,-0.231622
1916,1189,1189.227086,160.747926,1189.208186,1865.322724,64.537729,34.156127,10.853057,64.537729,1186,...,0.026749,8.025200e+00,B-,100.000,364.5,False,True,1.752692,723.096134,0.185134


In [41]:
aux_3.columns

Index(['peaks', 'fwhm_centr', 'rough_sums', 'centroids', 'variances', 'pk_hei',
       'lf_thr', 'rg_thr', 'promns', 'lf_bas', 'rg_bas', 'widths', 'wi_hei',
       'lf_ips', 'rg_ips', 'engy_pk_det', 'nuclide_name', 'energy', 'unc_en',
       'intensity', 'unc_i', 'half_life', 'decay', 'decay_%', 'key_gamma',
       'is_key_gamma', 'is_to_consider', 'delta_en', 'recalib_engy_ransac',
       'new_delta_en'],
      dtype='object')

In [42]:
new_ch_en_calib

Polynomial([ 1.22153806e+03,  9.82635880e+02, -1.31252172e-01], domain=[ 396.42677778, 3614.15831169], window=[-1.,  1.], symbol='x')

In [43]:
# 2023-Set-01 PAREI AQUI

In [44]:
pks_comprehensive_df.columns

Index(['peaks', 'fwhm_centr', 'rough_sums', 'centroids', 'variances', 'pk_hei',
       'lf_thr', 'rg_thr', 'promns', 'lf_bas', 'rg_bas', 'widths', 'wi_hei',
       'lf_ips', 'rg_ips', 'engy_pk_det'],
      dtype='object')

In [45]:
# def add_engy_to_pks_df(peaks_net_kev_df, func_en):
#     peaks_net_kev_df['engy_pk_det'] = func_en(peaks_net_kev_df.centroids)

In [46]:
# 2023-Sep-7 Include recalibrated energy

In [47]:
pks_comprehensive_df['recalib_energy']=new_ch_en_calib(pks_comprehensive_df.centroids)

In [48]:
pks_comprehensive_df

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,rg_bas,widths,wi_hei,lf_ips,rg_ips,engy_pk_det,recalib_energy
0,30,29.785567,1893.942368,29.963695,14133.214332,818.646195,201.465549,450.836680,818.646195,28,33,2.244704,409.323097,28.663214,30.907919,16.318529,14.882194
1,128,128.364887,2876.788495,127.767831,11283.211505,720.585627,359.256221,71.539300,720.585627,100,137,2.761022,360.292813,126.984376,129.745398,76.080802,74.636461
2,145,144.459911,1278.607977,144.896569,10161.329960,389.571554,4.276963,247.018556,389.571554,142,151,2.657272,194.785777,143.131275,145.788547,86.547152,85.101309
3,236,235.273124,462.759320,235.202199,12019.150830,123.425784,40.769738,84.205044,123.425784,233,240,2.919528,61.712892,233.813360,236.732888,141.727537,140.273282
4,396,396.401754,630.310441,396.426778,10404.641509,162.845302,70.909899,49.239266,162.845302,392,404,3.333009,81.422651,394.735249,398.068258,240.242260,238.770932
5,489,488.997063,249.976688,488.967375,4072.023312,125.089174,88.241458,88.978919,125.089174,485,493,1.411704,62.544587,488.291211,489.702915,296.788299,295.305981
6,581,581.463638,430.396173,581.647407,5405.821287,147.276132,89.934593,1.069452,147.276132,579,585,2.564867,73.638066,580.181204,582.746071,353.419539,351.925344
7,716,715.508125,222.738094,715.666805,4033.829966,79.006167,15.639623,46.454617,79.006167,713,719,2.684467,39.503084,714.165892,716.850359,435.310799,433.797888
8,841,841.967813,2900.206925,841.976062,7873.573309,559.895371,213.926360,11.088422,559.895371,836,850,4.639239,279.947685,839.648194,844.287433,512.490852,510.958635
9,960,960.088168,483.773366,960.065118,3335.841582,148.624963,51.372759,40.677023,148.624963,955,963,2.878815,74.312482,958.648761,961.527576,584.648031,583.096300


In [49]:
# 2023-Jun-20: Cálculo das atividades

In [50]:
# 2023-Jun-14
# Loading calibration pkl file:
calib_pkl_name = 'data/f100_gmx_2021.pkl'
calib_df = pd.read_pickle(calib_pkl_name)
calib_df

,Engy,Effic,sEffic,effic_func
0,59.54,6.4299,3.61,743
1,81.00,5.6528,3.61,743
2,276.40,1.6306,3.61,"((0.5110491663154504, -1.542825320223859, 0.42..."
3,302.85,1.5006,3.61,"((0.5110491663154504, -1.542825320223859, 0.42..."
4,356.02,1.4023,3.61,743
5,383.85,1.3216,3.61,743
6,604.69,0.9112,3.61,743
7,661.66,0.7900,3.61,743
8,795.84,0.7000,3.61,743
9,1155.00,0.5916,0.01,743


In [51]:
p_eff, _ = calib_df.effic_func[3]

In [52]:
def en_eff(engy):
    return np.exp(p_eff(np.log(engy)))

In [53]:
en_eff(1332)

0.5341252964668152

In [54]:
# 2023-Aug-18
# AQUI escolher nucludeo que tiver mais picos identificados (Ag-110m tem mais que Cs-137)
# Por enquanto, usar a calibracao robusta, que deve bastar.

In [55]:
cross_df = pd.merge(pks_comprehensive_df, nucl_iear1_selctd_gamms_df, how='cross')
cross_df["delta_en"] = cross_df.recalib_energy - cross_df.energy
df_result = pd.DataFrame(cross_df.loc[np.abs(cross_df.delta_en) < en_toler_ident])
df_result

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,...,unc_en,intensity,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma,is_to_consider,delta_en
543,396,396.401754,630.310441,396.426778,10404.641509,162.845302,70.909899,49.239266,162.845302,392,...,0.002,43.60000,0.5307178534777213,1.062200e+01,B-,100.000,238.6,True,True,0.138932
667,489,488.997063,249.976688,488.967375,4072.023312,125.089174,88.241458,88.978919,125.089174,485,...,0.002,18.47088,0.11249527437185973,2.706000e+01,B-,100.000,351.9,False,True,0.081981
786,581,581.463638,430.396173,581.647407,5405.821287,147.276132,89.934593,1.069452,147.276132,579,...,0.0021,35.72370,0.2352736185805795,2.706000e+01,B-,100.000,351.9,True,True,-0.006656
1129,960,960.088168,483.773366,960.065118,3335.841582,148.624963,51.372759,40.677023,148.624963,955,...,0.002,85.00000,0.3,3.053000e+00,B-,100.000,583.2,True,True,-0.090700
1260,1003,1002.888165,769.886444,1002.931019,3458.113556,220.739274,40.375893,77.703349,220.739274,990,...,0.007,45.44000,0.19,1.971000e+01,B-,99.979,609.3,True,True,-0.039338
1428,1082,1082.208995,2125.483990,1082.200436,4570.516010,657.235129,251.432911,123.634783,657.235129,1073,...,0.0011,95.61000,0.09,2.498300e+02,B-,98.670,657.8,True,True,-0.055762
1566,1088,1088.722180,401.081225,1088.105023,2556.918775,97.223236,59.126652,28.889179,69.221096,1086,...,0.003,85.10000,0.2,3.008000e+01,B-,100.000,661.7,True,True,-0.345907
1665,1115,1114.774649,176.695976,1115.022855,1999.057501,67.734059,24.189260,36.845929,67.734059,1112,...,0.0012,10.70000,0.05,2.498300e+02,B-,98.670,657.8,False,True,0.132280
1785,1163,1162.050038,336.295814,1161.943753,2591.560129,85.795902,0.176165,73.885618,85.795902,1152,...,0.0015,16.69000,0.07,2.498300e+02,B-,98.670,657.8,False,True,-0.260342
1916,1189,1189.227086,160.747926,1189.208186,1865.322724,64.537729,34.156127,10.853057,64.537729,1186,...,0.005,1.76855,0.026749,8.025200e+00,B-,100.000,364.5,False,True,0.159063


In [56]:
lv_time=a_spec.lv_time
# Sample size (L, kg, g etc.)
samp_size = 0.1
sam_descr = a_spec.sam_descr
source_datetime = a_spec.source_datetime
start_datetime = a_spec.start_datetime

In [57]:
df_result["disintegr"] = df_result.rough_sums / (en_eff(df_result.energy) * 1e-4 * df_result.intensity)
df_result['activity_conc'] = df_result.disintegr / (lv_time*samp_size)
df_result

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,...,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma,is_to_consider,delta_en,disintegr,activity_conc
543,396,396.401754,630.310441,396.426778,10404.641509,162.845302,70.909899,49.239266,162.845302,392,...,0.5307178534777213,1.062200e+01,B-,100.000,238.6,True,True,0.138932,7.324243e+04,14.648486
667,489,488.997063,249.976688,488.967375,4072.023312,125.089174,88.241458,88.978919,125.089174,485,...,0.11249527437185973,2.706000e+01,B-,100.000,351.9,False,True,0.081981,8.502770e+04,17.005540
786,581,581.463638,430.396173,581.647407,5405.821287,147.276132,89.934593,1.069452,147.276132,579,...,0.2352736185805795,2.706000e+01,B-,100.000,351.9,True,True,-0.006656,8.929519e+04,17.859037
1129,960,960.088168,483.773366,960.065118,3335.841582,148.624963,51.372759,40.677023,148.624963,955,...,0.3,3.053000e+00,B-,100.000,583.2,True,True,-0.090700,6.280187e+04,12.560374
1260,1003,1002.888165,769.886444,1002.931019,3458.113556,220.739274,40.375893,77.703349,220.739274,990,...,0.19,1.971000e+01,B-,99.979,609.3,True,True,-0.039338,1.925320e+05,38.506404
1428,1082,1082.208995,2125.483990,1082.200436,4570.516010,657.235129,251.432911,123.634783,657.235129,1073,...,0.09,2.498300e+02,B-,98.670,657.8,True,True,-0.055762,2.655030e+05,53.100602
1566,1088,1088.722180,401.081225,1088.105023,2556.918775,97.223236,59.126652,28.889179,69.221096,1086,...,0.2,3.008000e+01,B-,100.000,661.7,True,True,-0.345907,5.650064e+04,11.300127
1665,1115,1114.774649,176.695976,1115.022855,1999.057501,67.734059,24.189260,36.845929,67.734059,1112,...,0.05,2.498300e+02,B-,98.670,657.8,False,True,0.132280,2.009797e+05,40.195943
1785,1163,1162.050038,336.295814,1161.943753,2591.560129,85.795902,0.176165,73.885618,85.795902,1152,...,0.07,2.498300e+02,B-,98.670,657.8,False,True,-0.260342,2.517501e+05,50.350025
1916,1189,1189.227086,160.747926,1189.208186,1865.322724,64.537729,34.156127,10.853057,64.537729,1186,...,0.026749,8.025200e+00,B-,100.000,364.5,False,True,0.159063,1.151661e+06,230.332119


In [58]:
df_result.loc[df_result.nuclide_name=="110ag"]

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,...,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma,is_to_consider,delta_en,disintegr,activity_conc
1428,1082,1082.208995,2125.483990,1082.200436,4570.516010,657.235129,251.432911,123.634783,657.235129,1073,...,0.09,249.83,B-,98.67,657.8,True,True,-0.055762,265503.009753,53.100602
1665,1115,1114.774649,176.695976,1115.022855,1999.057501,67.734059,24.189260,36.845929,67.734059,1112,...,0.05,249.83,B-,98.67,657.8,False,True,0.132280,200979.715396,40.195943
1785,1163,1162.050038,336.295814,1161.943753,2591.560129,85.795902,0.176165,73.885618,85.795902,1152,...,0.07,249.83,B-,98.67,657.8,False,True,-0.260342,251750.124341,50.350025
2023,1256,1256.302992,405.106118,1256.297717,2257.095859,134.196651,62.170938,26.847523,134.196651,1253,...,0.07,249.83,B-,98.67,657.8,False,True,0.108798,234901.997231,46.980399
2379,1454,1453.734014,1344.932670,1453.640093,3338.139458,367.505693,29.182944,101.857014,367.505693,1449,...,1.1,249.83,B-,98.67,657.8,False,True,-0.084441,256618.596357,51.323719
2616,1540,1540.041676,709.425445,1539.865159,2764.574555,175.614136,63.155763,38.866820,175.614136,1534,...,0.3,249.83,B-,98.67,657.8,False,True,-0.223803,300342.441376,60.068488
3797,2272,2271.692429,360.400194,2271.722928,921.547003,83.645558,7.173466,18.857337,83.645558,2267,...,0.5,249.83,B-,98.67,657.8,False,True,-0.032783,277092.023485,55.418405
4035,2470,2469.534452,183.362379,2469.277794,470.637621,47.058621,3.040656,22.974186,47.058621,2460,...,0.15,249.83,B-,98.67,657.8,False,True,-0.115772,284945.009912,56.989002


In [59]:
# 2023-Ago-4 PAREI AQUI. Fazer a indexação para o cálculo da atividade considerando a média das linhas de cada nuclídeo
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#dataframe-column-selection-in-groupby
d5_grouped = df_result.groupby("nuclide_name")[['energy','intensity','centroids','rough_sums','variances','disintegr']]
# d5_grouped = d5.groupby("nuclide_name")[['disintegr']]
# sdfsdf = pd.DataFrame([[d5_grouped.mean(), d5_grouped.median()]]) 
# sdfsdf


In [61]:
df_result.groupby("nuclide_name")[['disintegr']].describe()

disintegr                                             \
                 count          mean            std           min   
nuclide_name                                                        
110ag              8.0  2.590166e+05   31069.618102  2.009797e+05   
131i               1.0  1.151661e+06            NaN  1.151661e+06   
137cs              1.0  5.650064e+04            NaN  5.650064e+04   
208Tl              1.0  6.280187e+04            NaN  6.280187e+04   
212Pb              1.0  7.324243e+04            NaN  7.324243e+04   
214Bi              7.0  5.245558e+05  213825.210072  1.925320e+05   
214Pb              2.0  8.716144e+04    3017.569182  8.502770e+04   
228Ac              3.0  5.157420e+05  321437.066494  3.294775e+05   
24na               1.0  3.266259e+04            NaN  3.266259e+04   
40k                1.0  6.071168e+06            NaN  6.071168e+06   
54mn               1.0  3.873793e+04            NaN  3.873793e+04   
58co               1.0  3.539912e+05            NaN  3.539912e+05   
60co               2.0  1.095084e+06     447.153265  1.094768e+06   
65zn               1.0  1.561311e+05            NaN  1.561311e+05   

                                                                      
                       25%           50%           75%           max  
nuclide_name                                                          
110ag         2.475381e+05  2.610608e+05  2.790553e+05  3.003424e+05  
131i          1.151661e+06  1.151661e+06  1.151661e+06  1.151661e+06  
137cs         5.650064e+04  5.650064e+04  5.650064e+04  5.650064e+04  
208Tl         6.280187e+04  6.280187e+04  6.280187e+04  6.280187e+04  
212Pb         7.324243e+04  7.324243e+04  7.324243e+04  7.324243e+04  
214Bi         4.055789e+05  5.201526e+05  6.597345e+05  8.285796e+05  
214Pb         8.609457e+04  8.716144e+04  8.822831e+04  8.929519e+04  
228Ac         3.301606e+05  3.308437e+05  6.088742e+05  8.869047e+05  
24na          3.266259e+04  3.266259e+04  3.266259e+04  3.266259e+04  
40k           6.071168e+06  6.071168e+06  6.071168e+06  6.071168e+06  
54mn          3.873793e+04  3.873793e+04  3.873793e+04  3.873793e+04  
58co          3.539912e+05  3.539912e+05  3.539912e+05  3.539912e+05  
60co          1.094926e+06  1.095084e+06  1.095242e+06  1.095400e+06  
65zn          1.561311e+05  1.561311e+05  1.561311e+05  1.561311e+05

In [78]:
f_name

PosixPath('/home/jovyan/data/some_spectra/SI06122.Chn')

In [79]:
results_pkl_file=f_name.with_stem(f_name.stem+'_result').with_suffix('.pkl')

In [80]:
df_result.to_pickle(results_pkl_file)

In [81]:
read_result_df = pd.read_pickle(results_pkl_file)
read_result_df

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,...,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma,is_to_consider,delta_en,disintegr,activity_conc
543,396,396.401754,630.310441,396.426778,10404.641509,162.845302,70.909899,49.239266,162.845302,392,...,0.5307178534777213,1.062200e+01,B-,100.000,238.6,True,True,0.138932,7.324243e+04,14.648486
667,489,488.997063,249.976688,488.967375,4072.023312,125.089174,88.241458,88.978919,125.089174,485,...,0.11249527437185973,2.706000e+01,B-,100.000,351.9,False,True,0.081981,8.502770e+04,17.005540
786,581,581.463638,430.396173,581.647407,5405.821287,147.276132,89.934593,1.069452,147.276132,579,...,0.2352736185805795,2.706000e+01,B-,100.000,351.9,True,True,-0.006656,8.929519e+04,17.859037
1129,960,960.088168,483.773366,960.065118,3335.841582,148.624963,51.372759,40.677023,148.624963,955,...,0.3,3.053000e+00,B-,100.000,583.2,True,True,-0.090700,6.280187e+04,12.560374
1260,1003,1002.888165,769.886444,1002.931019,3458.113556,220.739274,40.375893,77.703349,220.739274,990,...,0.19,1.971000e+01,B-,99.979,609.3,True,True,-0.039338,1.925320e+05,38.506404
1428,1082,1082.208995,2125.483990,1082.200436,4570.516010,657.235129,251.432911,123.634783,657.235129,1073,...,0.09,2.498300e+02,B-,98.670,657.8,True,True,-0.055762,2.655030e+05,53.100602
1566,1088,1088.722180,401.081225,1088.105023,2556.918775,97.223236,59.126652,28.889179,69.221096,1086,...,0.2,3.008000e+01,B-,100.000,661.7,True,True,-0.345907,5.650064e+04,11.300127
1665,1115,1114.774649,176.695976,1115.022855,1999.057501,67.734059,24.189260,36.845929,67.734059,1112,...,0.05,2.498300e+02,B-,98.670,657.8,False,True,0.132280,2.009797e+05,40.195943
1785,1163,1162.050038,336.295814,1161.943753,2591.560129,85.795902,0.176165,73.885618,85.795902,1152,...,0.07,2.498300e+02,B-,98.670,657.8,False,True,-0.260342,2.517501e+05,50.350025
1916,1189,1189.227086,160.747926,1189.208186,1865.322724,64.537729,34.156127,10.853057,64.537729,1186,...,0.026749,8.025200e+00,B-,100.000,364.5,False,True,0.159063,1.151661e+06,230.332119


In [82]:
read_result_df.groupby("nuclide_name")[['disintegr']].describe()

disintegr                                             \
                 count          mean            std           min   
nuclide_name                                                        
110ag              8.0  2.590166e+05   31069.618102  2.009797e+05   
131i               1.0  1.151661e+06            NaN  1.151661e+06   
137cs              1.0  5.650064e+04            NaN  5.650064e+04   
208Tl              1.0  6.280187e+04            NaN  6.280187e+04   
212Pb              1.0  7.324243e+04            NaN  7.324243e+04   
214Bi              7.0  5.245558e+05  213825.210072  1.925320e+05   
214Pb              2.0  8.716144e+04    3017.569182  8.502770e+04   
228Ac              3.0  5.157420e+05  321437.066494  3.294775e+05   
24na               1.0  3.266259e+04            NaN  3.266259e+04   
40k                1.0  6.071168e+06            NaN  6.071168e+06   
54mn               1.0  3.873793e+04            NaN  3.873793e+04   
58co               1.0  3.539912e+05            NaN  3.539912e+05   
60co               2.0  1.095084e+06     447.153265  1.094768e+06   
65zn               1.0  1.561311e+05            NaN  1.561311e+05   

                                                                      
                       25%           50%           75%           max  
nuclide_name                                                          
110ag         2.475381e+05  2.610608e+05  2.790553e+05  3.003424e+05  
131i          1.151661e+06  1.151661e+06  1.151661e+06  1.151661e+06  
137cs         5.650064e+04  5.650064e+04  5.650064e+04  5.650064e+04  
208Tl         6.280187e+04  6.280187e+04  6.280187e+04  6.280187e+04  
212Pb         7.324243e+04  7.324243e+04  7.324243e+04  7.324243e+04  
214Bi         4.055789e+05  5.201526e+05  6.597345e+05  8.285796e+05  
214Pb         8.609457e+04  8.716144e+04  8.822831e+04  8.929519e+04  
228Ac         3.301606e+05  3.308437e+05  6.088742e+05  8.869047e+05  
24na          3.266259e+04  3.266259e+04  3.266259e+04  3.266259e+04  
40k           6.071168e+06  6.071168e+06  6.071168e+06  6.071168e+06  
54mn          3.873793e+04  3.873793e+04  3.873793e+04  3.873793e+04  
58co          3.539912e+05  3.539912e+05  3.539912e+05  3.539912e+05  
60co          1.094926e+06  1.095084e+06  1.095242e+06  1.095400e+06  
65zn          1.561311e+05  1.561311e+05  1.561311e+05  1.561311e+05

In [84]:
read_result_df.loc[df_result.nuclide_name=="110ag"]

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,...,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma,is_to_consider,delta_en,disintegr,activity_conc
1428,1082,1082.208995,2125.483990,1082.200436,4570.516010,657.235129,251.432911,123.634783,657.235129,1073,...,0.09,249.83,B-,98.67,657.8,True,True,-0.055762,265503.009753,53.100602
1665,1115,1114.774649,176.695976,1115.022855,1999.057501,67.734059,24.189260,36.845929,67.734059,1112,...,0.05,249.83,B-,98.67,657.8,False,True,0.132280,200979.715396,40.195943
1785,1163,1162.050038,336.295814,1161.943753,2591.560129,85.795902,0.176165,73.885618,85.795902,1152,...,0.07,249.83,B-,98.67,657.8,False,True,-0.260342,251750.124341,50.350025
2023,1256,1256.302992,405.106118,1256.297717,2257.095859,134.196651,62.170938,26.847523,134.196651,1253,...,0.07,249.83,B-,98.67,657.8,False,True,0.108798,234901.997231,46.980399
2379,1454,1453.734014,1344.932670,1453.640093,3338.139458,367.505693,29.182944,101.857014,367.505693,1449,...,1.1,249.83,B-,98.67,657.8,False,True,-0.084441,256618.596357,51.323719
2616,1540,1540.041676,709.425445,1539.865159,2764.574555,175.614136,63.155763,38.866820,175.614136,1534,...,0.3,249.83,B-,98.67,657.8,False,True,-0.223803,300342.441376,60.068488
3797,2272,2271.692429,360.400194,2271.722928,921.547003,83.645558,7.173466,18.857337,83.645558,2267,...,0.5,249.83,B-,98.67,657.8,False,True,-0.032783,277092.023485,55.418405
4035,2470,2469.534452,183.362379,2469.277794,470.637621,47.058621,3.040656,22.974186,47.058621,2460,...,0.15,249.83,B-,98.67,657.8,False,True,-0.115772,284945.009912,56.989002
